In [3]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time

ModuleNotFoundError: No module named 'tensorflow'

In [5]:

from xgboost import XGBClassifier
from xgboost import XGBRegressor

# Synthetic

In [9]:
import numpy as np
np.random.seed(42)

def draw_sequence(length, digit_low, digit_high, counter_low, counter_high):
    digits, counter, important_digits = [], [], []

    while len(digits) < length:
        max_count = np.random.randint(counter_low + 1, counter_high + 1)
        counter_i = np.arange(max_count, counter_low - 1, -1)
        counter.extend(counter_i)

        digits_i = np.random.randint(digit_low, digit_high + 1, len(counter_i))
        if len(digits_i) + len(digits) <= length:
            important_digits.append(digits_i[-1])
        digits.extend(digits_i)

    digits = digits[:length]
    counter = counter[:length]
    label = sum(important_digits)

    return digits, counter, label

sequence_length = 10
digit_low = 0
digit_high = 2
counter_low = 0
counter_high = 2

draw_sequence(sequence_length, digit_low, digit_high, counter_low, counter_high)
train_size = 50000
test_size = 10000

def generate_dataset(size):
    digits_list = []
    counter_list = []
    labels = []
    for _ in range(size):
        digits, counter, label = draw_sequence(sequence_length, digit_low, digit_high, counter_low, counter_high)
        digits_list.append(digits)
        counter_list.append(counter)
        labels.append(label)
    return np.array(digits_list), np.array(counter_list), np.array(labels)

X_train_digits, X_train_counter, y_train = generate_dataset(train_size)
X_test_digits, X_test_counter, y_test = generate_dataset(test_size)

np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_digits.npy', X_train_digits)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_counter.npy', X_train_counter)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_labels.npy', y_train)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_digits.npy', X_test_digits)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_counter.npy', X_test_counter)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_labels.npy', y_test)

In [6]:
import numpy as np

In [7]:
X_train_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_digits.npy')
X_train_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_counter.npy')
y_train = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_labels.npy')
X_test_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_digits.npy')
X_test_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_counter.npy')
y_test = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_labels.npy')

In [8]:
y_train = np.eye(np.max(y_train) + 1)[y_train]
y_test = np.eye(np.max(y_test) + 1)[y_test]

In [9]:
X_train = np.concatenate([X_train_digits, X_train_counter], axis=1)
X_test = np.concatenate([X_test_digits, X_test_counter], axis=1)

In [ ]:
# xgb 
masksper = 512
d = X_train.shape[1]
X_class = np.concatenate([X_train]*masksper, 0)
Y_class = np.concatenate([y_train]*masksper, 0)
B = np.concatenate(
[np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(d)], 1, keepdims=True) for _ in range(X_class.shape[0])],
1)
B = np.float32(B.T)
X_class = np.concatenate((X_class*B, B), 1)

In [ ]:
# Train classifier
from xgboost import XGBClassifier
est = XGBClassifier(n_estimators=256, device='gpu')
est.fit(X_class, Y_class)

In [ ]:
# save model to path
path = '/work/users/d/d/ddinh/aaco/models/'
est.save_model(path + 'synthetic_raw.model')

In [ ]:
Bval = np.concatenate(
  [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(8)], 1, keepdims=True) for _ in range(X_test.shape[0])],
  1)
Bval = np.float32(Bval.T)
Xvalmasked = np.concatenate((X_test*Bval, Bval), 1)

val_preds = est.predict_proba(Xvalmasked)

print(np.mean(np.round(val_preds)==y_test))

# mean number of masks Bval
print(np.mean(np.sum(Bval, axis=1)))

In [1]:
import numpy as np

# Load training data
X_train_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_digits.npy')
X_train_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_counter.npy')
y_train = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_labels.npy')

# Load test data
X_test_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_digits.npy')
X_test_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_counter.npy')
y_test = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_labels.npy')

print("X_train_digits.shape =", X_train_digits.shape)     # (N, 10)
print("X_train_counter.shape =", X_train_counter.shape)   # (N, 10)
print("y_train.shape =", y_train.shape)                   # (N,)

print("X_test_digits.shape =", X_test_digits.shape)       # (N, 10)
print("X_test_counter.shape =", X_test_counter.shape)     # (N, 10)
print("y_test.shape =", y_test.shape)                     # (N,)

N_train = X_train_digits.shape[0]  # Number of training samples
N_test = X_test_digits.shape[0]    # Number of test samples
seq_length = X_train_digits.shape[1]  # Should be 10


X_train_digits.shape = (50000, 10)
X_train_counter.shape = (50000, 10)
y_train.shape = (50000,)
X_test_digits.shape = (10000, 10)
X_test_counter.shape = (10000, 10)
y_test.shape = (10000,)


In [2]:
def convert_to_mnist_images(X_digits, digit2images):
    N = X_digits.shape[0]
    seq_length = X_digits.shape[1]
    
    X_images = np.zeros((N, seq_length, 28, 28), dtype=np.float32)
    
    for i in range(N):
        for t in range(seq_length):
            digit = X_digits[i, t]
            img_tensor = random.choice(digit2images[digit])
            img_np = img_tensor.squeeze().numpy()  
            X_images[i, t] = img_np
    
    return X_images


In [3]:
import torch
from torchvision import datasets, transforms
import random

transform = transforms.Compose([transforms.ToTensor()])

mnist_train = datasets.MNIST(
    root='./mnist_data',
    train=True,
    download=True,
    transform=transform
)

digit2images = {d: [] for d in range(10)}

for i in range(len(mnist_train)):
    img, label = mnist_train[i]      
    digit2images[label].append(img)  


In [4]:
X_train_digits_images = convert_to_mnist_images(X_train_digits, digit2images)
X_train_counter_images = convert_to_mnist_images(X_train_counter, digit2images)

print("X_train_digits_images.shape =", X_train_digits_images.shape)   # (N_train, 10, 28, 28)
print("X_train_counter_images.shape =", X_train_counter_images.shape)

X_test_digits_images = convert_to_mnist_images(X_test_digits, digit2images)
X_test_counter_images = convert_to_mnist_images(X_test_counter, digit2images)

print("X_test_digits_images.shape =", X_test_digits_images.shape)     # (N_test, 10, 28, 28)
print("X_test_counter_images.shape =", X_test_counter_images.shape)   


X_train_digits_images.shape = (50000, 10, 28, 28)
X_train_counter_images.shape = (50000, 10, 28, 28)
X_test_digits_images.shape = (10000, 10, 28, 28)
X_test_counter_images.shape = (10000, 10, 28, 28)


In [5]:
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/train_digits_images.npy', X_train_digits_images)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/train_counter_images.npy', X_train_counter_images)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/train_labels.npy', y_train)

np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/test_digits_images.npy', X_test_digits_images)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/test_counter_images.npy', X_test_counter_images)
np.save('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/test_labels.npy', y_test)

In [10]:
X_test_digits_images.shape

(10000, 10, 28, 28)

In [17]:
def reshape_time_to_channels(X):
    """
    X: shape (N, T, H, W) -> (N, H, W, T)
    """
    X = X.transpose((0, 2, 3, 1))  
    return X


In [33]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class RandomMask(Layer):
    def __init__(self):
        super(RandomMask, self).__init__()
        self.dim = 10  # Number of time steps (channels)
        
    def call(self, inputs):
        # Get the dynamic batch size
        batch_size = tf.shape(inputs)[0]  # Dynamically get the batch size

        # Generate a random slice index for each sample in the batch
        slice_indices = tf.random.uniform(
            shape=(batch_size,), minval=1, maxval=self.dim + 1, dtype=tf.int32
        )

        # Create a batch of random permutation matrices (one for each sample)
        eye = tf.eye(self.dim)  # Identity matrix (dim x dim)
        permuted_eyes = tf.random.shuffle(tf.tile(eye[None, :, :], [batch_size, 1, 1]))  # (batch_size, dim, dim)

        # Create the masks by summing over the random slices
        masks = tf.map_fn(
            lambda x: tf.reduce_sum(x[0][:, :x[1]], axis=1, keepdims=True),
            (permuted_eyes, slice_indices),
            fn_output_signature=tf.float32
        )  # Shape: (batch_size, dim, 1)

        # Reshape and broadcast masks to match the input shape
        masks = tf.transpose(masks, perm=[0, 2, 1])  # Shape: (batch_size, 1, 1, dim)
        masks = tf.expand_dims(masks, axis=1)  # Shape: (batch_size, 1, 1, dim)
        masks = tf.broadcast_to(masks, tf.shape(inputs))  # Shape: (batch_size, height, width, dim)

        # Apply the mask to the inputs
        return inputs * masks

    def get_config(self):
        config = super().get_config().copy()
        return config

In [19]:
from tensorflow.keras import layers
from tensorflow.keras import models

def resnet_block(x, filters, kernel_size=3, strides=1):
    """
    A simple ResNet block with:
    Conv -> BN -> ReLU -> Conv -> BN -> Add (skip) -> ReLU
    """
    shortcut = x

    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, kernel_size, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    if strides != 1:
        shortcut = layers.Conv2D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    return x


def build_resnet_branch(input_shape=(28,28,10), mask_rate=0.3):
    """
    Builds a small ResNet-like model that accepts input of shape (H=28, W=28, channels=10).
    Returns: (input_tensor, output_tensor)
    """
    input_tensor = layers.Input(shape=input_shape)

    # Optionally mask the input at training time
    x = RandomMask()(input_tensor)

    # Initial projection
    x = layers.Conv2D(32, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # ResNet blocks
    x = resnet_block(x, filters=32, kernel_size=3, strides=1)  # block 1
    x = resnet_block(x, filters=64, kernel_size=3, strides=2)  # block 2, downsample
    x = resnet_block(x, filters=64, kernel_size=3, strides=1)  # block 3
    x = resnet_block(x, filters=128, kernel_size=3, strides=2) # block 4, downsample

    # Flatten
    x = layers.GlobalAveragePooling2D()(x)  # or Flatten() + Dense, etc.
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    return input_tensor, x

def build_multimodal_resnet(input_shape=(28,28,10),
                            mask_rate=0.3,
                            num_classes=np.max(y_train)+1):
    """
    - input_shape: e.g. (28,28,10)
    - mask_rate: probability for random masking
    - regression: if True, final Dense(1, 'linear')
                  else classification with Dense(num_classes, 'softmax').

    Returns: a compiled Keras Model
    """
    # Digits branch
    digits_input, digits_features = build_resnet_branch(input_shape, mask_rate)

    # Counter branch
    counter_input, counter_features = build_resnet_branch(input_shape, mask_rate)

    # Combine
    combined = layers.Concatenate()([digits_features, counter_features])
    x = layers.Dense(128, activation='relu')(combined)


    # Classification
    if num_classes is None:
        raise ValueError("Must specify num_classes for classification")
    output = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=[digits_input, counter_input], outputs=output)
    model.compile(optimizer='adam', 
                    loss='sparse_categorical_crossentropy', 
                    metrics=['accuracy'])
    return model


In [34]:
X_train_digits_raw = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/train_digits_images.npy')
X_train_counter_raw = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/train_counter_images.npy')
y_train = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/train_labels.npy')

X_test_digits_raw  = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/test_digits_images.npy')
X_test_counter_raw = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/test_counter_images.npy')
y_test = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_images/test_labels.npy')

X_train_digits  = reshape_time_to_channels(X_train_digits_raw)    
X_train_counter = reshape_time_to_channels(X_train_counter_raw)
X_test_digits   = reshape_time_to_channels(X_test_digits_raw)
X_test_counter  = reshape_time_to_channels(X_test_counter_raw)

from sklearn.model_selection import train_test_split
X_train_digits, X_val_digits, X_train_counter, X_val_counter, y_train_split, y_val_split = \
    train_test_split(X_train_digits, X_train_counter, y_train, test_size=0.2, random_state=42)

print("Train digits shape:", X_train_digits.shape)       # ~80% of original
print("Train counter shape:", X_train_counter.shape)
print("Val digits shape:", X_val_digits.shape)           # ~20% of original
print("Val counter shape:", X_val_counter.shape)
print("y_train_split shape:", y_train_split.shape)
print("y_val_split shape:", y_val_split.shape)


model = build_multimodal_resnet(
    input_shape=(28,28,10),
    mask_rate=0.3,        # random masking rate
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath='/work/users/d/d/ddinh/aaco/models/synthetic_images.keras',
    monitor='val_loss',          # Monitor validation loss
    mode='min',                  # Minimize the monitored value
    save_best_only=True,         # Save only the best model
    verbose=1
)


history = model.fit(
    x=[X_train_digits, X_train_counter],
    y=y_train_split,
    epochs=30,
    batch_size=256,
    validation_data=([X_val_digits, X_val_counter], y_val_split),
    callbacks=[checkpoint_cb],
    verbose=1
)

print("\nEvaluating on Test set:")
test_loss = model.evaluate([X_test_digits, X_test_counter], y_test, verbose=1)
print("Test Loss:", test_loss)



Train digits shape: (40000, 28, 28, 10)
Train counter shape: (40000, 28, 28, 10)
Val digits shape: (10000, 28, 28, 10)
Val counter shape: (10000, 28, 28, 10)
y_train_split shape: (40000,)
y_val_split shape: (10000,)
Epoch 1/30
  1/157 ━━━━━━━━━━━━━━━━━━━━ 1:13:27 28s/step - accuracy: 0.0234 - loss: 2.6393

: 